# CREATE PINECONE INDEX

In [1]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [2]:
pc.create_index(
    name="newdataindex",
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [3]:
import PyPDF2
import re
PDFFile = open("oracle.pdf",'rb')
PDF = PyPDF2.PdfReader(PDFFile)
pages = len(PDF.pages)
key = '/Annots'
uri = '/URI'
ank = '/A'

url_list = []

for page in range(8,11):
    print("Current Page: {}".format(page))
    pageSliced = PDF.pages[page]
    pageObject = pageSliced.get_object()
    if key in pageObject.keys():
        ann = pageObject[key]
        for a in ann:
            u = a.get_object()
            if uri in u[ank].keys():
                #print(u[ank][uri])
                url_list.append(u[ank][uri])

Current Page: 8
Current Page: 9
Current Page: 10


In [ ]:
# remove unecessary urls

In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nest_asyncio

nest_asyncio.apply()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

documents_list = []
page_list = []

loader = WebBaseLoader(url_list)
loader.requests_per_second = 2
documents_list = loader.aload()

len(documents_list)

Fetching pages: 100%|##########| 89/89 [00:06<00:00, 14.33it/s]


89

In [20]:
for document in documents_list:
    document.page_content = document.page_content.replace("\n", " ")
    document.page_content = document.page_content.replace("PreviousNext JavaScript must be enabled to correctly display this content", " ")

documents_list

[Document(page_content=" PJB_BILL_TYPE_CLASSES_B                \t\tTables and Views for Project ManagementPJB_BILL_TYPE_CLASSES_BPJB_BILL_TYPE_CLASSES_B   This table will be used to store the bill type classifications. This is the seeded data. Details   Schema: FUSION   Object owner: PJB   Object type: TABLE   Tablespace: REFERENCE    Primary Key   Name  Columns     PJB_BILL_TYPE_CLASSES_B_PK   BILL_TYPE_CLASSIFICATION_CODE     Columns   Name Datatype Length Precision Not-null Comments    BILL_TYPE_CLASSIFICATION_CODE VARCHAR2 30  Yes Classification of the Billing method. *****   REVENUE_FLAG VARCHAR2 1   Revenue process use this method for create revenue if enable *****   INVOICE_FLAG VARCHAR2 1   Invoice process use this method for create revenue if enable *****   OBJECT_VERSION_NUMBER NUMBER  9 Yes Used to implement optimistic locking. This number is incremented every time that the row is updated. The number is compared at the start and end of a transaction to detect whether anothe

In [21]:
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceHubEmbeddings, OpenAIEmbeddings
import os
from dotenv import load_dotenv
import asyncio
load_dotenv()

openai_embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

index_name = "newdataindex"

# pinecone = PineconeVectorStore.afrom_documents(documents_list[0:1000], openai_embeddings, index_name=index_name)
pinecone = PineconeVectorStore.afrom_documents(documents_list[-45:], openai_embeddings, index_name=index_name)

asyncio.run(main=asyncio.create_task(pinecone))

In [48]:
print(len(page_list))

144
